In [1]:
%matplotlib inline

from __future__ import division

import numpy as np
import gym
import logz
import scipy.signal
import os
import torch as tc
import time
import inspect
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from itertools import izip, imap

import model_utils as K
import crayon

from collections import namedtuple
from IPython import display


%load_ext autoreload
%autoreload 2

## Constants

In [2]:
def dict_to_nt(dictionary):
    return namedtuple('GenericDict', dictionary.keys())(**dictionary)

In [3]:
SAMPLE_CONFIG = dict_to_nt(
    dict(
        exp_name='test1',
        env_name='CartPole-v0',
        n_iter=500,
        gamma=1.0,
        min_timesteps_per_batch=1000,
        max_path_length=None,
        learning_rate=5e-3,
        reward_to_go=True,
        animate=True,
        logdir=None,
        normalize_advantages=True,
        nn_baseline=False,
        n_hidden_layers=1,
        hidden_dim=32,
        seed=0,
    ),
)
SAMPLE_CONFIG

GenericDict(n_iter=500, exp_name='test1', learning_rate=0.005, min_timesteps_per_batch=1000, env_name='CartPole-v0', seed=0, reward_to_go=True, logdir=None, normalize_advantages=True, n_hidden_layers=1, nn_baseline=False, animate=True, max_path_length=None, gamma=1.0, hidden_dim=32)

## Utilities

In [4]:
ModelOpt = namedtuple('ModelOpt', ['model', 'optimizer'])


def pathlength(path):
    return len(path.rewards)

In [5]:
def build_mlp(
    D_in, 
    D_out,
    n_hidden_layers=2, 
    H=64, 
    activation=tc.nn.LeakyReLU,
    output_activation=None,
):
    layers = (
        [tc.nn.Linear(D_in, H), activation()] +
        ([tc.nn.Linear(H, H), activation()] * (n_hidden_layers - 1)) +
        [tc.nn.Linear(H, D_out)]
    )
    if output_activation is not None:
        layers.append(output_activation())
    return tc.nn.Sequential(*layers)

## POLICY GRADIENT

### Setup env

In [6]:
EnvDetails = namedtuple(
    'EnvDetails',
    ['env', 'discrete', 'max_path_length', 'ob_dim', 'axn_dim'],
)

def setup_env(env_name, max_path_length):
    # Make the gym environment
    env = gym.make(env_name)
    discrete = isinstance(env.action_space, gym.spaces.Discrete)
    return EnvDetails(
        env=env,
        # Is this env continuous, or discrete?
        discrete=discrete,
        # Maximum length for episodes
        max_path_length=max_path_length or env.spec.max_episode_steps,
        # Observation and action sizes
        ob_dim=env.observation_space.shape[0],
        axn_dim=env.action_space.n if discrete else env.action_space.shape[0],
    )

### Models

#### Discrete MLP model

In [7]:
DiscModelOut = namedtuple(
    'DiscModelOut',
    ['axn', 'axn_probas', 'log_axn_probas', 'chosen_axn_log_proba'],
)

class DiscreteModel(tc.nn.Module):
    
    def __init__(self, D_in, D_out, hidden_dim, n_hidden_layers, activation=tc.nn.LeakyReLU, lr=1e-3):
        super(self.__class__, self).__init__()
        self.mlp_model = build_mlp(
            output_activation=tc.nn.Softmax,
            D_in=D_in,
            D_out=D_out,
            H=hidden_dim,
            n_hidden_layers=n_hidden_layers,
            activation=activation,
        )
        self.is_discrete = True
        self.optimizer = None
        self.lr = lr
        
    def forward(self, obs, **kwargs):
        # obs -> (batch_sz, obs_dim)
        # axn_probas -> (batch_sz, axns_dim)
        axn_probas = self.mlp_model(K.array_to_variable(obs))
        # axn -> (batch_sz, 1)
        axn = axn_probas.multinomial()
        # axn_log_proba
        log_axn_probas = tc.log(axn_probas)
        return DiscModelOut(
            axn=axn,
            axn_probas=axn_probas,
            log_axn_probas=tc.log(axn_probas),
            chosen_axn_log_proba=log_axn_probas.gather(1, axn.view(-1,1)),
        )
    
    def update_params(self, paths, reward_to_go, nn_baseline, gamma=1.):
        """Assumes the rewards following the terminal state is 0."""
        if not reward_to_go:
            raise NotImplementedError
        optimizer = self._get_optimizer()
        rewards = compute_loss_rtg(paths=paths, gamma=gamma, nn_baseline=True)
        max_pathlen = get_max_pathlen(paths)
        loss = 0
        for i, p in enumerate(paths):
            model_outs = self.forward(p.obs)
            chosen_log_probas = model_outs.log_axn_probas.gather(1, K.array_to_variable(p.axns).long().view(-1, 1))
            loss -= (chosen_log_probas * K.array_to_variable(rewards[i,:len(p)])).sum()
        optimizer.zero_grad()
        loss.backward()
        # utils.clip_grad_norm(self.parameters(), 40)
        optimizer.step()
        return loss.data[0]
        
    def _get_optimizer(self):
        if self.optimizer is not None:
            return self.optimizer
        else:
            self.optimizer = tc.optim.Adam(params=self.parameters(), lr=self.lr)
            return self.optimizer


def model_out_to_axn(model_out):
    return model_out.axn.data[0][0]


def compute_loss_rtg(paths, gamma, nn_baseline):
    num_paths = len(paths)
    max_pathlen = get_max_pathlen(paths)
    # dim (num_paths, max_episode_len)
    path_returns = np.zeros((num_paths, max_pathlen))
    #     print (
    #         'num_paths: {p}, max_pathlen: {m}'.format(
    #             p=num_paths,
    #             m=max_pathlen,
    #         )
    #     )
    for i, p in enumerate(paths):
        path_returns[i] = _compute_returns_rtg(
            path=p,
            gamma=gamma,
            max_pathlen=max_pathlen,
        )
    if nn_baseline:
        path_returns = (
            (path_returns - path_returns.mean(axis=0)) /
            (path_returns.std(axis=0) + np.finfo(np.float32).eps)
        )
    return path_returns


def get_max_pathlen(paths):
    return max(imap(len, paths))

def _compute_returns_rtg(path, gamma, max_pathlen):
    """Returns reward to go returns for each time step"""
    Rs = np.zeros((1, max_pathlen))
    R = 0
    for i in reversed(xrange(len(path))):
        R = path.rewards[i] + gamma * R
        Rs[0, i] = R
    return Rs


def extract_model_out_field(path, field):
    return map(lambda mo: getattr(mo, field), path.model_outs)


def extract_chosen_axn_log_proba(path, pad_to_size=None):
    var = tc.cat(extract_model_out_field(path=path, field='chosen_axn_log_proba'), dim=1)
    if not pad_to_size or pad_to_size==var.size()[1]:
        return var
    return tc.cat([var, tc.zeros(1, pad_to_size - len(path.model_outs))],dim=1)

#### Continuous MLP

In [8]:
class ContinousModel(tc.nn.Module):
    
    def __init__(self, D_in, D_out, hidden_dim, n_hidden_layers, activation=tc.nn.LeakyReLU, lr=1e-3):
        super(self.__class__, self).__init__()
        self.mu = build_mlp(
            output_activation=None,
            D_in=D_in,
            D_out=D_out,
            H=hidden_dim,
            n_hidden_layers=n_hidden_layers,
            activation=activation,
        )
        # Assume a diagonal covariance
        self.z = K.CudableVariable(tc.randn(1, D_out)).float()
        self.is_discrete = False
        self.optimizer = None
        self.lr = lr

    def forward(self, obs, **kwargs):
        # obs -> (batch_sz, obs_dim)
        # axn_probas -> (batch_sz, axns_dim)
        mu = self.mu(K.array_to_variable(obs))
        # axn -> (batch_sz, 1)
        axn = mu + (self.z * tc.normal(K.placeholder(mu.size())))
        # axn_log_proba
        return dict_to_nt(dict(axn=axn, mu=mu))
    
    def update_params(self, paths, reward_to_go, nn_baseline, gamma=1.):
        """Assumes the rewards following the terminal state is 0."""
        if not reward_to_go:
            raise NotImplementedError
        optimizer = self._get_optimizer()
        rewards = compute_loss_rtg(paths=paths, gamma=gamma, nn_baseline=True)
        max_pathlen = get_max_pathlen(paths)
        loss = 0
        for i, p in enumerate(paths):
            model_outs = self.forward(p.obs)
            loss += (
                0.5 *
                (1 / tc.pow(self.z, 2)) *
                tc.pow( (K.array_to_variable(p.axns) - model_outs.axn), 2) *
                K.array_to_variable(rewards[i,:len(p)])
            ).sum()
        optimizer.zero_grad()
        loss.backward()
        # utils.clip_grad_norm(self.parameters(), 40)
        optimizer.step()
        return loss.data[0]
        
    def _get_optimizer(self):
        if self.optimizer is not None:
            return self.optimizer
        else:
            self.optimizer = tc.optim.Adam(params=self.parameters(), lr=self.lr)
            return self.optimizer

In [9]:
def create_model(env_details, n_hidden_layers, hidden_dim, lr=1e-3):
    if not env_details.discrete:
        return ContinousModel(
            D_in=env_details.ob_dim,
            D_out=env_details.axn_dim,
            hidden_dim=hidden_dim,
            n_hidden_layers=n_hidden_layers,
            lr=lr,
        )
    else:
        return DiscreteModel(
            D_in=env_details.ob_dim,
            D_out=env_details.axn_dim,
            hidden_dim=hidden_dim,
            n_hidden_layers=n_hidden_layers,
            lr=lr,
        )

### Simulations

In [10]:
def show_state(env, step=0, info='', render_img=None):
    """Show cart pole inline.
    
    Source: https://stackoverflow.com/a/45179251/1950328
    """
    plt.figure(3)
    if render_img is None:
        render_img = plt.imshow(env.render(mode='rgb_array'))
    else:
        render_img.set_data(env.render(mode='rgb_array')) # just update the data
    plt.title("%s | Step: %d %s" % (env.spec.id, step, info))
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    return render_img

In [11]:
class Path(namedtuple('Path', ['obs', 'rewards', 'axns', 'model_outs'])):
    
    def __len__(self):
        return len(self.rewards)


def rollout(model, env_details, animate, min_timesteps_per_batch):
    total_timesteps = 0
    env = env_details.env
    timesteps_this_batch = 0
    paths = []
    render_img = None
    while True:
        ob = env.reset()
        obs, axns, rewards, model_outs = [], [], [], []
        animate_this_episode=(animate)
        steps = 0
        while True:
            if animate_this_episode:
                render_img = show_state(
                    env=env,
                    step=steps,
                    render_img=render_img,
                )
                time.sleep(0.05)
            obs.append(ob)
            model_out = model(ob[None])
            model_outs.append(model_out)
            axn = model_out_to_axn(model_out=model_out)
            axns.append(axn)
            ob, rew, done, _ = env.step(axn)
            rewards.append(rew)
            steps += 1
            if done or steps > env_details.max_path_length:
                break
        path = Path(
            obs=np.array(obs), 
            rewards=np.array(rewards), 
            axns=np.array(axns),
            model_outs=model_outs,
        )
        paths.append(path)
        timesteps_this_batch += pathlength(path)
        if timesteps_this_batch > min_timesteps_per_batch:
            break
    total_timesteps += timesteps_this_batch
    env_details.env.close()
    return paths

### Model training

In [14]:
def compute_reward_per_episode(paths):
    return sum(imap(lambda p: p.rewards.sum(), paths)) / len(paths)


def avg_path_len(paths):
    return sum(imap(len, paths)) / len(paths)


def train_env(env_name, expt_name, config=SAMPLE_CONFIG):
    env_details = setup_env(env_name=env_name, max_path_length=config.max_path_length)
    print 'Env_details:', env_details
    model = create_model(
        env_details=env_details,
        n_hidden_layers=config.n_hidden_layers,
        hidden_dim=config.hidden_dim,
    )
    print 'Constructed Model: ', model
    expt = crayon.get_experiment(name=env_name)
    pbar = K.Progbar(target=config.n_iter)
    for i in xrange(config.n_iter):
        paths = rollout(
            model=model,
            env_details=env_details,
            animate=False,
            min_timesteps_per_batch=config.min_timesteps_per_batch,
        )
        loss = model.update_params(
            paths=paths,
            reward_to_go=True,
            nn_baseline=True,
            gamma=config.gamma,
        )
        # Losses
        expt.add_scalar_dict(
            {
                'loss': loss,
                'reward_per_episode': compute_reward_per_episode(paths),
                'avg_path_len': avg_path_len(paths),
            },
        )
        pbar.update(i)
    return dict_to_nt(dict(env_details=env_details, model=model))


def sim_trained_model(env_details, model, min_timesteps_per_batch=200):
    rollout(
        model=model,
        env_details=env_details,
        animate=True,
        min_timesteps_per_batch=min_timesteps_per_batch,
    )

### Core code

In [13]:
#crayon.clear_expts()

#### Cart-Pole

In [16]:
model_cp = train_env(
    env_name='CartPole-v0',
    expt_name='cart_pole_pg',
    config=dict_to_nt(
        dict(
            n_iter=500,
            gamma=0.99,
            min_timesteps_per_batch=1000,
            max_path_length=None,
            learning_rate=5e-3,
            reward_to_go=True,
            nn_baseline=True,
            n_hidden_layers=1,
            hidden_dim=32,
        ),
    ),
)

Env_details: EnvDetails(env=<TimeLimit<CartPoleEnv<CartPole-v0>>>, discrete=True, max_path_length=200, ob_dim=4, axn_dim=2)
Constructed Model:  DiscreteModel (
  (mlp_model): Sequential (
    (0): Linear (4 -> 32)
    (1): LeakyReLU (0.01)
    (2): Linear (32 -> 2)
    (3): Softmax ()
  )
)
499/500 [============================>.] - ETA: 0s